# Explorando tendencias de mercado de Airbnb

La ciudad de Nueva York tiene una variedad de anuncios de Airbnb para satisfacer la alta demanda de alojamiento temporal para viajeros, con diferentes niveles de precios, tipos de habitaciones y ubicaciones.

La ciudad de Nueva York, es una de las ciudades más visitadas del mundo. Hay muchos anuncios de Airbnb en la ciudad de Nueva York para satisfacer la alta demanda de alojamiento temporal para viajeros, que puede durar desde unas pocas noches hasta muchos meses. En este proyecto, analizaremos más de cerca el mercado de Airbnb de Nueva York combinando datos de varios tipos de archivos como .csv, .tsv y .xlsx.

Recuerde que los archivos CSV, TSV y Excel son tres formatos comunes para almacenar datos. Tiene a su disposición [tres archivos](https://tajamar365.sharepoint.com/:f:/s/3405-MasterIA2024-2025/EoXMgjAJLLNJqWDyKDig8kABSUkpWj_HGQjl267qNnHS9g?e=foxjFc) que contienen datos sobre los anuncios de Airbnb de 2019:  

- `airbnb_price.csv `: Este es un archivo CSV que contiene datos sobre precios y ubicaciones de anuncios de Airbnb.  
  1. `listing_id`: identificador único del anuncio  
  2. `price`: precio del anuncio por noche en USD  
  3. `nbhood_full`: nombre del distrito y del barrio donde se encuentra el anuncio  
    

- `airbnb_room_type.xlsx`: Este es un archivo Excel que contiene datos sobre las descripciones de los anuncios de Airbnb y los tipos de habitaciones.  
  1. `listing_id`: identificador único del anuncio  
  2. `description`: descripción del anuncio     
  3. `room_type`: Airbnb tiene tres tipos de habitaciones: habitaciones compartidas, habitaciones privadas y casas o apartamentos completos


- `airbnb_last_review.tsv`: Este es un archivo TSV que contiene datos sobre los nombres de los anfitriones de Airbnb y las fechas de revisión.  
  1. `listing_id`: identificador único del anuncio  
  2. `host_name`:  nombre del anfitrión del anuncio  
  3. `last_review`: echa en la que se revisó el anuncio por última vez   

In [1]:
from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder.appName("AirBNB").getOrCreate()

In [2]:
from google.colab import files

# Subir los archivos CSV
uploaded = files.upload()

Saving airbnb_last_review.tsv to airbnb_last_review.tsv
Saving airbnb_price.csv to airbnb_price.csv
Saving airbnb_room_type.xlsx to airbnb_room_type.xlsx


In [3]:
!pip install openpyxl


In [4]:
import pandas as pd

df_tsv = spark.read.option("delimiter", "\t").option("header", "true").csv("airbnb_last_review.tsv")
df_csv = spark.read.option("header", "true").csv("airbnb_price.csv")
df_excel = spark.createDataFrame(pd.read_excel("airbnb_room_type.xlsx", engine="openpyxl"))


Como consultor que trabaja para una empresa inmobiliaria emergente, ha recopilado datos de anuncios de Airbnb de varias fuentes para investigar el mercado de alquileres a corto plazo en Nueva York. Analizará estos datos para brindar información sobre habitaciones privadas a la empresa inmobiliaria.

- ¿Cuáles son las fechas de las primeras y las últimas reseñas? Almacene estos valores como dos variables independientes con los nombres que prefiera.

In [16]:
import pyspark.sql.functions as F

df = df_tsv.join(df_csv, on="listing_id").join(df_excel, on="listing_id")

In [27]:
df.show()

df.printSchema()

+----------+----------------+---------------+-----------+--------------------+--------------------+---------------+
|listing_id|       host_name|    last_review|      price|         nbhood_full|         description|      room_type|
+----------+----------------+---------------+-----------+--------------------+--------------------+---------------+
|      2595|        Jennifer|    May 21 2019|225 dollars|  Manhattan, Midtown|Skylit Midtown Ca...|Entire home/apt|
|      3831|     LisaRoxanne|   July 05 2019| 89 dollars|Brooklyn, Clinton...|Cozy Entire Floor...|Entire home/apt|
|      5099|           Chris|   June 22 2019|200 dollars|Manhattan, Murray...|Large Cozy 1 BR A...|Entire home/apt|
|      5178|        Shunichi|   June 24 2019| 79 dollars|Manhattan, Hell's...|Large Furnished R...|   private room|
|      5238|             Ben|   June 09 2019|150 dollars|Manhattan, Chinatown|Cute & Cozy Lower...|Entire home/apt|
|      5295|            Lena|   June 22 2019|135 dollars|Manhattan, Uppe

In [23]:
# Definimos dos variables para la fecha más próxima y la más lejana
# Convertimos los datos a fecha, y luego las ponemos en formato MMM dd yyyy(Apr 09 2019)
last_review = F.date_format(F.max(F.to_date(F.col("last_review"), "MMMM dd yyyy")), "MMM dd yyyy")
earlier_review = F.date_format(F.min(F.to_date(F.col("last_review"), "MMMM dd yyyy")), "MMM dd yyyy")

# guardamos la consulta de máximos y mínimos en variables
df_last_review = df.select(last_review.alias("most_recent_review"))
df_earlier_review = df.select(earlier_review.alias("first_review"))

# mostramos los resultados
df_last_review.show()
df_earlier_review.show()

+------------------+
|most_recent_review|
+------------------+
|       Jul 09 2019|
+------------------+

+------------+
|first_review|
+------------+
| Jan 01 2019|
+------------+



Como vemos, nos da los valores máximos y mínimos de la columna *last_review*, formateado tal y como sale en la tabla

- ¿Cuántos de los anuncios son habitaciones privadas? Guárdelo en cualquier variable.

In [25]:
# filtramos por private_room y contamos las filas
private_room = df.where(F.col("room_type") == "Private room").count()

#imprimimos el resultado
print(private_room)

7241


Hay 7241 anuncios que son habitaciones privadas

- ¿Cuál es el precio promedio de los anuncios? Redondee a los dos decimales más cercanos y guárdelo en una variable.

In [28]:
# hacemos un cast a la columna price para quedarnos solo con el precio en formato double
price = F.split(F.col("price"), " ")[0].cast("double")
df = df.withColumn("price", price)

# calculamos la media
avg_price = df.select(F.round(F.avg(F.col("price")), 2).alias("average_price")).collect()[0][0]

# imprimimos el resultado
print(avg_price)

141.78


El precio medio de los anuncios es de 141.78 dólares




- Combine las nuevas variables en un DataFrame llamado review_dates con cuatro columnas en el siguiente orden: first_reviewed, last_reviewed, nb_private_rooms y avg_price. El DataFrame solo debe contener una fila de valores.

In [30]:
# extraemos los valores de las variables guardadas como dataframes
firts_reviewed = df_earlier_review.collect()[0][0]
last_reviewed = df_last_review.collect()[0][0]

# específicamos las columnas y los datos que va a tener nuestro dataframe (1 fila)
columns = ["first_reviewed", "last_reviewed", "private_room", "avg_price"]
data = [[firts_reviewed, last_reviewed, private_room, avg_price]]

# creamos el dataframe
review_dates = spark.createDataFrame(data, columns)

# imprimimos los resultados
review_dates.show()

+--------------+-------------+------------+---------+
|first_reviewed|last_reviewed|private_room|avg_price|
+--------------+-------------+------------+---------+
|   Jan 01 2019|  Jul 09 2019|        7241|   141.78|
+--------------+-------------+------------+---------+

